In [1]:
# import essential files from /src folder
import sys
import os

sys.path.append(os.path.abspath('../src'))

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score , classification_report #calculate accuracy
from hog import preprocessing
import matplotlib.pyplot as plt

In [ ]:
train_path = "D:\GitHub\FER-with-HOG-and-SVM\datasets/train"     #change your path here
data_train, label_train, data_test, label_test = preprocessing(train_path)

In [ ]:
def _transform_data(data, labels):
    #Generate input array X
    X = np.array(data)

    #Generate output array y
    lbe = LabelEncoder()
    lbe.fit(labels)
    y = lbe.transform(labels)

    return X, y

In [ ]:
#poly kernel
X_train , y_train = _transform_data(data_train, label_train)
X_test , y_test = _transform_data(data_test, label_test)

In [ ]:
model = svm.SVC(kernel='poly', degree = 3, gamma= 0.1,C=75, coef0=1)
model.fit(X_train,y_train)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction.shape

In [ ]:
rp = metrics.classification_report(y_test, prediction, target_names=['anger', 'happy', 'neutral', 'sad', 'suprise'],output_dict=True)
rp = pd.DataFrame(rp).transpose()
rp.drop("support",axis=1,inplace=True)
rp.drop("macro avg",axis=0,inplace=True)
rp.drop("weighted avg",axis=0,inplace=True)
rp.drop("accuracy",axis=0,inplace=True)
rp = round(rp.iloc[0:]*100,2)
rp

# # note on metrics:
# # precision: how often the model is correct when it predicts a class (true positives / (true positives + false positives))
# high value => when model predicts a class, it is likely to be correct
# # recall: how often the model predicts a class when it is actually that class (true positives / (true positives + false negatives))
# high value => when the class is present, the model is likely to predict it
# # f1: harmonic mean of precision and recall, useful when you want to balance both metrics

In [ ]:
def show_plot(title,data):
    face = ['anger', 'happy', 'neutral', 'sad', 'suprise']

    # Create a bar chart
    bars = plt.bar(face, data, color=['red', 'lightgreen', 'yellow', 'gray', 'plum'])

    # Add labels and title
    plt.xlabel('Expression')
    plt.ylabel('Percentage (%)')
    plt.title(title)

    for bar in bars:
        val = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, val, round(val, 2), ha='center', va='bottom')
    # Show the bar chart
    plt.show()

In [ ]:
show_plot("Precision",rp.precision)
show_plot("Recall",rp.recall)
show_plot("F1 Score", rp['f1-score'])

In [ ]:
confusion_Matrix = confusion_matrix(y_test, prediction)
confusion_Matrix

In [ ]:
confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
acc_per_class = confusion_matrix.diagonal()
apc = pd.DataFrame(index=['anger', 'happy', 'neutral', 'sad', 'suprise'],data=acc_per_class,columns=['Accuracy per class'])
apc = round(apc.iloc[0:]*100,2)
show_plot("Accuracy per class", apc['Accuracy per class'])

In [ ]:
print("Accuracy: ", metrics.accuracy_score(y_test, prediction)*100)

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [True, True])
cm_display.plot()
plt.show()

#from 0 to 4: anger, happy, neutral, sad, suprise